# ¿Qué es la regresión lineal?

La **regresión lineal** es un modelo estadístico que predice el valor de una **variable continua** (ejemplo: precio de una casa) usando una o más **variables independientes** (ejemplo: área, número de habitaciones).

## Concepto clave
Modela la relación entre variables mediante una **línea recta** descrita por la ecuación:

$Y = \beta_0 + \beta_1X + \epsilon$

- $Y$: Variable dependiente (objetivo).
- $X$: Variable independiente (predictora).
- $\beta_0$: Intercepto (valor de $Y$ cuando $X = 0$.
- $\beta_1$: Pendiente (cambio en $Y$ por unidad de cambio en $X$).
- $\epsilon$: Error (diferencia entre el valor real y el predicho).

## Objetivo
Encontrar los valores de $\beta_0$ y $\beta_1$ que **minimicen el error** entre los datos reales y la línea ajustada (método de *mínimos cuadrados*).

## Ejemplo
Predecir el **precio de una casa** $Y$ en función de su **área** $X$.

## Aplicaciones
- Predicción de ventas, precios, tendencias.
- Análisis de impacto entre variables.
- Base para modelos avanzados de *machine learning*.

# Carga de librerías

In [300]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carga de archivo

In [301]:
# Fuente: https://www.kaggle.com/datasets/cesaregr/medelln-properties
# Columnas para cargar
columnas =['neighbourhood', 'property_type', 'price',
       'rooms', 'baths', 'area','age', 'garages',
       'stratum']
dfPropiedades = pd.read_csv("./medellin_properties.csv",usecols=columnas)

# Carga de validación y limpieza de datos 

In [302]:
# Validamos los tipos de datos
dfPropiedades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   neighbourhood  9999 non-null   object 
 1   property_type  9999 non-null   object 
 2   price          9999 non-null   int64  
 3   rooms          9936 non-null   float64
 4   baths          9877 non-null   float64
 5   area           9999 non-null   float64
 6   age            7362 non-null   float64
 7   garages        6793 non-null   object 
 8   stratum        9999 non-null   int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 703.2+ KB


In [303]:
dfPropiedades["garages"].value_counts()

garages
1            4655
2            1626
4             190
3             168
5              45
6              36
10             23
8              20
Más de 10      17
7               8
9               5
Name: count, dtype: int64

In [304]:
# Reemplazamos el valor de garages de texto a valor
dfPropiedades.loc[dfPropiedades["garages"]=="Más de 10","garages"]=15

In [305]:
# Columnas en nulo
dfPropiedades.isna().sum()

neighbourhood       0
property_type       0
price               0
rooms              63
baths             122
area                0
age              2637
garages          3206
stratum             0
dtype: int64

In [306]:
# Borramos los registros con rooms y baths en nulo
dfPropiedades.dropna(subset=["rooms","baths"],inplace=True)

# Asignar como cero los valores de age y garages
dfPropiedades.fillna(0,inplace=True)

# Convertimos todos los campos categóricos a mayúsculas
dfPropiedades[['neighbourhood','property_type']]=dfPropiedades[['neighbourhood','property_type']].astype(str).apply(lambda col: col.str.upper())

In [307]:
# Cambiamos el tipo de datos de garages a int
dfPropiedades["garages"]=dfPropiedades["garages"].astype(int)


In [308]:
dfPropiedades[['price','rooms', 'baths', 'area', 'age', 'garages','stratum']].min()

price      1600000.0
rooms            1.0
baths            1.0
area             0.0
age              0.0
garages          0.0
stratum          0.0
dtype: float64

In [309]:
# Descartamos registros con área = 0
dfPropiedades=dfPropiedades[dfPropiedades["area"]>0]

In [310]:
# neighbourhood (barrio) incluye mucho en el precio por lo que debemos analizarlo por aparte
# Contar la frecuencia de cada neighbourhood
cantidadRegBarrio = dfPropiedades['neighbourhood'].value_counts()
cantidadRegBarrio

neighbourhood
CALASANZ                333
LAS LOMITAS             211
LAURELES                210
EL POBLADO              194
SURAMERICA              158
                       ... 
SABANETA LA DOCTORA       1
TRANSVERSAL SUPERIOR      1
ENVIGADO LA PAZ LV        1
SENDERO BRUJO             1
SAN BERNANDO              1
Name: count, Length: 988, dtype: int64

In [311]:
barriosPocosReg=len(cantidadRegBarrio[cantidadRegBarrio<5])
totalReg= len(dfPropiedades)
porcentaje = barriosPocosReg/totalReg
print(f"Se encontraron {barriosPocosReg} valores de neighbourhood con menos de 5 registros. Esto corresponde al {porcentaje:0.2%} de los {totalReg} registros")

Se encontraron 642 valores de neighbourhood con menos de 5 registros. Esto corresponde al 6.52% de los 9844 registros


In [312]:

# Agrupar neighbourhoods con menos de 5 registros en "Otros"
dfPropiedades['neighbourhood'] = dfPropiedades['neighbourhood'].apply(
    lambda x: x if cantidadRegBarrio[x] >= 5 else 'OTROS'
)

In [313]:
dfPropiedades=dfPropiedades[dfPropiedades["neighbourhood"].isin(cantidadRegBarrio[cantidadRegBarrio>10].index)]

# Tratamiento de variables categóricas

## **One-Hot Encoding (La mejor opción para principiantes)**

### **¿Qué es?**
One-Hot Encoding transforma cada **categoría única** en una **columna binaria** (0 o 1).
- Ejemplo: Si tienes la categoría **"color"** con valores *"rojo"*, *"azul"* y *"verde"*, se crearán columnas como:
  - `is_azul` (1 si es azul, 0 si no)
  - `is_verde` (1 si es verde, 0 si no)
  *(Se omite `is_rojo` para evitar redundancia, usando `drop_first=True`)*

---

### **¿Por qué usarlo?**
* **No asume un orden artificial** (evita errores como pensar que "azul" > "rojo").
* **Funciona perfectamente con regresión lineal**.
* **Fácil de interpretar** en los resultados del modelo.

---

### **¿Cuándo NO usarlo?**
- Si la categoría **tiene un orden lógico** (ejemplo: *"primaria"*, *"secundaria"*, *"universidad"*).
  → En ese caso, usa **Ordinal Encoding** (asignar números: 1, 2, 3).

## **Categorías con Orden**
Si la categoría sí tiene un orden (ejemplo: nivel educativo: primaria, secundaria, universidad), usa Ordinal Encoding (asignar números según el orden: 1, 2, 3).

---
**Conclusión:**
Para regresión lineal, **One-Hot Encoding es la opción más simple, segura y recomendada** para variables categóricas sin orden. 😊


In [314]:
# Aplicamos One-Hot encoding ya que no hay orden en barrios ni en tipos de propiedad
dfPropiedades = pd.get_dummies(dfPropiedades, columns=['neighbourhood'], drop_first=True)
dfPropiedades = pd.get_dummies(dfPropiedades, columns=['property_type'], drop_first=True)

In [315]:
dfPropiedades

,price,rooms,baths,area,age,garages,stratum,neighbourhood_ALDEA VERDE,neighbourhood_ALQUERIA DE SAN ISIDRO,neighbourhood_ALTO DE LAS FLORES,...,neighbourhood_VEREDA SAN JOSE,neighbourhood_VILLA HERMOSA,neighbourhood_VILLA VERDE,neighbourhood_VIVIENDAS DEL SUR,neighbourhood_ZONA CENTRO,neighbourhood_ZONA RURAL,neighbourhood_ZUÑIGA,property_type_APARTAMENTO,property_type_CASA,property_type_PROYECTO
0,435000000,3.0,2.0,83.0,1.0,1,4,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,680000000,3.0,4.0,124.0,2.0,2,4,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,900000000,3.0,3.0,111.0,1.0,2,6,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,320000000,3.0,2.0,72.0,0.0,2,5,False,False,False,...,False,False,False,False,False,False,False,True,False,False
5,365000000,3.0,2.0,66.0,1.0,1,4,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,412100000,3.0,2.0,100.0,0.0,1,3,False,False,False,...,False,False,False,False,False,False,False,True,False,False
9994,685000000,3.0,3.0,94.0,1.0,2,5,False,False,False,...,False,False,False,False,False,False,False,True,False,False
9996,320000000,3.0,2.0,59.0,0.0,0,4,False,False,False,...,False,False,False,False,False,False,False,True,False,False
9997,450000000,4.0,3.0,147.0,3.0,1,3,False,False,False,...,False,False,False,False,False,False,False,True,False,False


# Creación de modelo de regresión lineal

In [316]:
dfPropiedades

,price,rooms,baths,area,age,garages,stratum,neighbourhood_ALDEA VERDE,neighbourhood_ALQUERIA DE SAN ISIDRO,neighbourhood_ALTO DE LAS FLORES,...,neighbourhood_VEREDA SAN JOSE,neighbourhood_VILLA HERMOSA,neighbourhood_VILLA VERDE,neighbourhood_VIVIENDAS DEL SUR,neighbourhood_ZONA CENTRO,neighbourhood_ZONA RURAL,neighbourhood_ZUÑIGA,property_type_APARTAMENTO,property_type_CASA,property_type_PROYECTO
0,435000000,3.0,2.0,83.0,1.0,1,4,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,680000000,3.0,4.0,124.0,2.0,2,4,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,900000000,3.0,3.0,111.0,1.0,2,6,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,320000000,3.0,2.0,72.0,0.0,2,5,False,False,False,...,False,False,False,False,False,False,False,True,False,False
5,365000000,3.0,2.0,66.0,1.0,1,4,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,412100000,3.0,2.0,100.0,0.0,1,3,False,False,False,...,False,False,False,False,False,False,False,True,False,False
9994,685000000,3.0,3.0,94.0,1.0,2,5,False,False,False,...,False,False,False,False,False,False,False,True,False,False
9996,320000000,3.0,2.0,59.0,0.0,0,4,False,False,False,...,False,False,False,False,False,False,False,True,False,False
9997,450000000,4.0,3.0,147.0,3.0,1,3,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [317]:
# Importamos train_test_split.
from sklearn.model_selection import train_test_split


# Creamos los dataset de características y objetivo.
X = dfPropiedades.drop(labels=["price"],axis=1)
y = dfPropiedades["price"]


# Split the dataset into training (80%) and testing (20%) sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [318]:
# Importamos LinearRegression
from sklearn.linear_model import LinearRegression

# Instanciamos el modelo regresión lineal
modelo = LinearRegression()

In [321]:
from sklearn.preprocessing import StandardScaler
# Scale the data.
scaler = StandardScaler()


X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [322]:
# Entrenamos el modelo
modelo.fit(X_train_scaled, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [323]:
# Hacemos predicciones en los set de prueba
y_pred = modelo.predict(X_test_scaled)

In [ ]:
# print(list(zip(y_test, y_pred)))

In [324]:
# Import metrics.
from sklearn.metrics import mean_squared_error, r2_score


# Calculate and print R^2 score.
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2:.4f}")

R-squared: -0.3713
